In [79]:
%pip install pandas matplotlib openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import re

#用pd读取input下的几个excel文件
df_cient_summary = pd.read_excel('./input/生意参谋_客户概况.xlsx')
df_follows_views = pd.read_excel('./input/生意参谋-客户关注分析-核心指标.xlsx')
df_follows_contents = pd.read_excel('./input/生意参谋-内容管理-关注.xlsx')
df_follows_contents_new = pd.read_excel('./input/生意参谋-客户关注分析-上新内容.xlsx')
df_follows_contents_list = pd.read_excel('./input/生意参谋-客户关注分析-清单内容.xlsx')
    

Note: you may need to restart the kernel to use updated packages.


In [81]:
# 删除哪些没有日期信息数据
df_follows_contents = df_follows_contents[df_follows_contents['关注内容列表'].str.contains(r'\d{4}-\d{2}-\d{2}', regex=True, na=False)]

#把 关注内容列表， 按照空行拆分为一个list，记住是空行
df_follows_contents['关注内容列表2'] = df_follows_contents['关注内容列表'].str.split('\n\t\n')

# 将拆分后的列表转换为 DataFrame
split_columns = df_follows_contents['关注内容列表2'].apply(pd.Series)



## 数据清理

In [87]:
#清理内容列表
#定义一个函数来提取数据
# def extract_data(text):
#     # 拆分文本为几个模块
#     sections = [section for section in text.split('\n\n')]
#     # 提取每个模块的标题和内容
#     title = sections[0].split('：')[1]

#     return {
#         "内容标题"：title,
# }


# # 示例数据
# data = [
#     "高级感衬衫！ | 衬衫怎么搭都好看，让你成为穿搭高手！\nID: 24255682137310\n多品上新\n\n2025-04-11\n20:11:12\n\n猜你喜欢已采纳\n\n已发布\n\n查看数据下线评论管理"
#     # 可以继续添加更多的内容
# ]

# # 创建DataFrame
# df = pd.DataFrame(data, columns=["内容"])

# 定义一个函数来提取信息
def extract_info(text):
    # 拆分文本，按空行分割
    sections = [section.strip() for section in text.split('\n\t\n')]
    #print(sections)
    # 提取各个模块内容
    title = sections[0].strip().split('\n')[0]  # 内容标题, 第一行
    content_id = re.search(r'ID:\s*(\d+)', sections[0]).group(1)  # id
    content_type = sections[0].strip().split('\n')[-1]  # 内容类型，最后一行
    publish_date_match = re.search(r'\d{4}-\d{2}-\d{2}', sections[1])  # 发布日期。 不能用位置判断，因为有字符“定时”
    if publish_date_match:
        publish_date = publish_date_match.group()  # 获取匹配的日期字符串
    else:
        publish_date = None  # 如果没有匹配，设置为 None
    
    publish_time = sections[1].strip().split('\n')[1]  # 发布时间
    adopted = sections[2].strip()  # 是否被采纳
    published = sections[3].strip()  # 是否已发布
    comments = sections[4].strip()  # 查看评论

    # 返回结果
    return {
        "内容标题": title,
        "id": content_id,
        "内容类型": content_type,
        "发布日期": publish_date,
        "发布时间": publish_time,
        "是否被采纳": adopted,
        "是否已发布": published,
        "查看评论": comments
    }

# 应用函数，提取所有行的信息
extracted_data = df_follows_contents["关注内容列表"].apply(extract_info)
print(len(extracted_data))

# 将提取到的数据转换为新的DataFrame
extracted_df = pd.DataFrame(extracted_data.tolist())
df_follows_contents = extracted_df

994
